In [ ]:
import pandas as pd
import re
import time
import numpy as np
from itertools import combinations


# THIS CELL IS ALL LAB CODE AND PREPROCESSING!!

# Load datasets
acm = pd.read_csv("ACM.csv")
dblp = pd.read_csv("DBLP2.csv", encoding="latin1") # breaks without this
perfect_mapping = pd.read_csv("DBLP-ACM_perfectMapping.csv")

# (a) Concat
def concatenate_record(rec):
    return " ".join(str(val) for val in rec.values)
    
def preprocess(text):
    # (b) lowercase
    text = text.lower()
    # (c) convert multiple spaces to one
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
    

# Preprocess 
for df in [acm, dblp]:
    for col in ['title', 'authors', 'venue']:
        if col != "id":
            df[col] = df[col].astype(str).apply(preprocess)

# Keep IDs separately
acm_ids = acm["id"].tolist()
dblp_ids = dblp["id"].tolist()


# Should we drop id here?
acm_strings = acm.drop('id', axis=1).apply(concatenate_record).tolist()
dblp_strings = dblp.drop('id', axis=1).apply(concatenate_record).tolist()

doc_ids = acm_ids + dblp_ids
doc_strings = acm_strings + dblp_strings

def get_minhash_arr(num_hashes:int,vocab:dict):
    """
    """
    length = len(vocab.keys())
    arr = np.zeros((num_hashes,length))
    for i in range(num_hashes):
        permutation = np.random.permutation(len(vocab.keys()))
        arr[i,:] = permutation.copy()
    return arr.astype(int)
def get_signature(minhash:np.ndarray, vector:np.ndarray):
    """
    """
    idx = np.nonzero(vector)[0].tolist()
    shingles = minhash[:,idx]
    signature = np.min(shingles,axis=1)
    return signature

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0

def compute_signature_similarity(signature_1, signature_2):
    """
    """
    # Ensure the matrices have the same shape
    if signature_1.shape != signature_2.shape:
        raise ValueError("Both signature matrices must have the same shape.")
    # Count the number of rows where the two matrices agree
    agreement_count = np.sum(signature_1 == signature_2)
    # Calculate the similarity
    similarity = agreement_count / signature_2.shape[0]

    return similarity

class LSH:
    """
    Implements the Locality Sensitive Hashing (LSH) technique for approximate
    nearest neighbor search.
    """
    buckets = []
    counter = 0

    def __init__(self, b: int):
        """
        Initializes the LSH instance with a specified number of bands.

        Parameters:
        - b (int): The number of bands to divide the signature into.
        """
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature: np.ndarray) -> np.ndarray:
        """
        Divides a given signature into subvectors based on the number of bands.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be divided.

        Returns:
        - np.ndarray: A stacked array where each row is a subvector of the signature.
        """
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature: np.ndarray):
        """
        Adds a signature to the appropriate LSH buckets based on its subvectors.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be hashed and added.
        """
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self) -> set:
        """
        Identifies candidate pairs from the LSH buckets that could be potential near duplicates.

        Returns:
        - set: A set of tuple pairs representing the indices of candidate signatures.
        """
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)


def shingle(text: str, k: int)->set:
    """
    Create a set of 'shingles' from the input text using k-shingling.

    Parameters:
        text (str): The input text to be converted into shingles.
        k (int): The length of the shingles (substring size).

    Returns:
        set: A set containing the shingles extracted from the input text.
    """
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)
def build_vocab(shingle_sets: list)->dict:
    """
    Constructs a vocabulary dictionary from a list of shingle sets.

    This function takes a list of shingle sets and creates a unified vocabulary
    dictionary. Each unique shingle across all sets is assigned a unique integer
    identifier.

    Parameters:
    - shingle_sets (list of set): A list containing sets of shingles.

    Returns:
    - dict: A vocabulary dictionary where keys are the unique shingles and values
      are their corresponding unique integer identifiers.

    Example:
    sets = [{"apple", "banana"}, {"banana", "cherry"}]
    build_vocab(sets)
    {'apple': 0, 'cherry': 1, 'banana': 2}  # The exact order might vary due to set behavior
    """
    full_set = {item for set_ in shingle_sets for item in set_}
    vocab = {}
    for i, shingle in enumerate(list(full_set)):
        vocab[shingle] = i
    return vocab
def one_hot(shingles: set, vocab: dict):
    vec = np.zeros(len(vocab))
    for shingle in shingles:
        idx = vocab[shingle]
        vec[idx] = 1
    return vec




In [7]:
# Shingling

k = 2
shingles = []
for sentence in doc_strings:
    shingles.append(shingle(sentence,k))
vocab = build_vocab(shingles)
print(f"Number of vocabulary is:{len(vocab)}")
shingles_1hot = []
for shingle_set in shingles:
    shingles_1hot.append(one_hot(shingle_set,vocab))
shingles_1hot = np.stack(shingles_1hot)


minhash_arr =  get_minhash_arr(200,vocab)
signatures = []
for vector in shingles_1hot:
    signatures.append(get_signature(minhash_arr,vector))
signatures = np.stack(signatures)
signatures.shape


b = 20   # number of buckets
lsh = LSH(b)
for signature in signatures:
    lsh.add_hash(signature)
candidate_pairs = lsh.check_candidates()
print(len(candidate_pairs))

candidates = []
for i, j in candidate_pairs:
    id1, id2 = doc_ids[i], doc_ids[j]

    # AI code, idk its broken
    if i < len(acm_ids) and j >= len(acm_ids):
        candidates.append((id1, id2))
    elif j < len(acm_ids) and i >= len(acm_ids):
        candidates.append((id2, id1))



perfect_set = set(zip(perfect_mapping["idACM"], perfect_mapping["idDBLP"]))
reported_set = set(candidates)

correct_matches = reported_set & perfect_set
precision = len(correct_matches) / len(reported_set) if reported_set else 0

print("Total ACM docs:", len(acm_ids))
print("Total DBLP docs:", len(dblp_ids))

print(f"LSH Precision: {precision:.4f}")
print(f"Candidate pairs: {len(reported_set)}")
print(f"Correct matches: {len(correct_matches)}")


Number of vocabulary is:1385
10
Total ACM docs: 2294
Total DBLP docs: 2616
LSH Precision: 0.0000
Candidate pairs: 0
Correct matches: 0


In [ ]:
print("ACM ID sample:", acm_ids[:5])
print("DBLP ID sample:", dblp_ids[:5])
print("Perfect mapping sample:", perfect_mapping.head())

# Check overlap
print("Any ACM IDs in mapping?", any(i in perfect_mapping["idACM"].values for i in acm_ids))
print("Any DBLP IDs in mapping?", any(i in perfect_mapping["idDBLP"].values for i in dblp_ids))



In [3]:
dblp_strings

["semantic integration of environmental models for application to global information systems and decision-making estimation of query-result distribution and its application in parallel-join load balancing incremental maintenance for non-distributive aggregate functions cost-based selection of path expression processing algorithms in object-oriented databases benchmarking spatial join operations with spatial output efficient geometry-based similarity search of 3d spatial databases mining the world wide web: an information search approach - book review enhanced abstract data types in object-relational databases report on dart '96: databases: active and real-time (concepts meet practice) unisql's next-generation object-relational database management system dual-buffering strategies in object bases aurora: a data stream management system priority assignment in real-time active databases wavecluster: a wavelet based clustering approach for spatial data in very large databases approximate qu